### Подключение данных

In [2]:
import pandas as pd

df = pd.read_csv('datasets/data_raw.csv')

### Описание данных
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита.

### Удаление и заполнение пропусков

In [48]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца income_type. Например, у человека с типом занятости сотрудник пропуск в столбце total_income должен быть заполнен медианным доходом среди всех записей с тем же типом.

In [49]:
for type in df['income_type'].unique():
    df.loc[(df['income_type'] == type) & (df['total_income'].isna()), 'total_income'] = \
    df.loc[(df['income_type'] == type), 'total_income'].median()

Заполните пропуски в столбце days_employed медианными значениями по каждого типа занятости income_type

In [50]:
for type in df['income_type'].unique():
    df.loc[(df['income_type'] == type) & (df['days_employed'].isna()), 'days_employed'] = \
    df.loc[(df['income_type'] == type), 'days_employed'].median()

In [51]:
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Аномалии. Выявление и корректировка

В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. таким артефактом будет отрицательное количество дней трудового стажа в столбце days_employed. Для реальных данных это нормально. Обработайте значения в этом столбце: замените все отрицательные значения положительными с помощью метода abs().

In [52]:
df['days_employed'] = df['days_employed'].abs()

Медианное значение трудового стажа days_employed в днях.

In [53]:
df.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

Выведите перечень уникальных значений столбца children

In [54]:
df['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

В столбце children есть два аномальных значения. Удалите строки, в которых встречаются такие аномальные значения из датафрейма.

In [55]:
df = df[(df['children'] != -1) & (df['children'] != 20)]

In [56]:
df['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### Обработка дубликатов

Обработайте неявные дубликаты в столбце education. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведите их к нижнему регистру. Проверьте остальные столбцы.

In [57]:
df['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [58]:
df['education'] = df['education'].str.lower()

In [59]:
df['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Выведите на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалите их.

In [60]:
df.duplicated().sum()

71

In [61]:
df = df.drop_duplicates()

In [62]:
df.duplicated().sum()

0

На основании диапазонов, указанных ниже, создайте в датафрейме data столбец total_income_category с категориями:
- 0–30000 — 'E';
- 30001–50000 — 'D';
- 50001–200000 — 'C';
- 200001–1000000 — 'B';
- 1000001 и выше — 'A'.

Например, кредитополучателю с доходом 25000 нужно назначить категорию 'E', а клиенту, получающему 235000, — 'B'. Используйте собственную функцию с именем categorize_income() и метод apply().

In [63]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [64]:
df['total_income_category'] = df['total_income'].apply(categorize_income)

In [65]:
df['total_income_category'].value_counts()

total_income_category
C    15921
B     5014
D      349
A       25
E       22
Name: count, dtype: int64

Выведите на экран перечень уникальных целей взятия кредита из столбца purpose.
Создайте функцию, которая на основании данных из столбца purpose сформирует новый столбец purpose_category, в который войдут следующие категории:
- 'операции с автомобилем',
- 'операции с недвижимостью',
- 'проведение свадьбы',
- 'получение образования'.

Например, если в столбце purpose находится подстрока 'на покупку автомобиля', то в столбце purpose_category должна появиться строка 'операции с автомобилем'.

Используйте собственную функцию с именем categorize_purpose() и метод apply(). Изучите данные в столбце purpose и определите, какие подстроки помогут вам правильно определить категорию.

In [66]:
df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [67]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [68]:
df['purpose_category'] = df['purpose'].apply(categorize_purpose)

In [69]:
df['purpose_category'].unique()

array(['операции с недвижимостью', 'операции с автомобилем',
       'получение образования', 'проведение свадьбы'], dtype=object)

### Сохранение обработанного датафрейма

In [71]:
df_csv = df.to_csv('datasets/data.csv', index = True) 

In [3]:
df = pd.read_csv('datasets/data.csv')

In [4]:
pivot_children = df.pivot_table(df, index = ['children'], values = 'debt', aggfunc = ['sum','count','mean'])
pivot_children

,sum,count,mean
,debt,debt,debt
children,,,
0,1063,14091,0.075438
1,444,4808,0.092346
2,194,2052,0.094542
3,27,330,0.081818
4,4,41,0.097561
5,0,9,0.000000
